# LISFLOOD - Monte Carlo Calibration Example
***

This notebook demonstrates a simple Monte Carlo-based calibration approach for LISFLOOD. </br>
It generates N random parameter sets within defined bounds and runs the model for each set.</br>
In the official GloFAS and EFAS versions, the calibration is of course not done with a simple random sampling, but a genetic algorithm called DEAP is used (see [Fortin et al., 2012](https://www.jmlr.org/papers/volume13/fortin12a/fortin12a.pdf)).</br>
* Technical details of the calibration tool itself can be found [here](https://github.com/ec-jrc/lisflood-calibration).
* EFAS-specific details, such as on parameters, skill and calibrated basins, you can get [here](https://confluence.ecmwf.int/display/CEMS/EFAS+v5.0+-+Calibration+Methodology+and+Data).
* The GloFAS counterpart you find [here](https://confluence.ecmwf.int/display/CEMS/GloFAS+v4+calibration+methodology+and+parameters).

**Author**: Timo Schaffhauser <br>
**Date**: 23rd February 2026<br>
***

## Parameters and their bounds
In the following we will quickly provide an overview about LISFLOOD's main calibration parameter.</br>
Mein calibration parameters here refers to the fact that those are (some) of the ones used in the official GloFAS and EFAS calibration strategies.</br>
However, as users potentially have noticed already throughout the work with the settings files, there are plenty of other parameters that c ould be changed. <br>
In a snow-affected catchment like the Aisen, people might want more control than just relying on the degree-day factor (*SnowMeltCoef*) and for example also adjust the melt temperature itself, which is not touched in the official GloFAS and EFAS calibrations. 
| ParameterName         | MinValue | MaxValue | DefaultValue |
|-----------------------|---------:|---------:|-------------:|
| UpperZoneTimeConstant  | 0.01     | 40       | 10           |
| LowerZoneTimeConstant  | 40       | 1000     | 100          |
| GwPercValue            | 0.01     | 2        | 0.8          |
| LZThreshold            | 0        | 30       | 10           |
| b_Xinanjiang           | 0.01     | 5        | 0.5          |
| PowerPrefFlow          | 0.5      | 8        | 4            |
| SnowMeltCoef           | 2.5      | 6.5      | 4            |
| CalChanMan1            | 0.5      | 2        | 1            |
| CalChanMan2            | 0.5      | 5        | 1            |
| GwLoss                 | 0        | 0.5      | 0            |
| TransSub               | 0        | 0.12     | 0            |

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import subprocess
import xml.etree.ElementTree as ET
import time

## Configuration

In [ ]:
# ============================================================
# CONFIGURATION - Change these values as needed
# ============================================================

# Number of Monte Carlo samples
N_SAMPLES = 10  # Can be changed to any number (e.g., 100, 1000)

# Paths
BASE_DIR = Path("/home/schafti/Documents/01_Hydrology/01_Lisflood/00_SourceCode/01_Playground/01_Test_Usecase/create_submodel_7859/7859_testing")
IN_DIR = BASE_DIR / "settings"
OUT_DIR = BASE_DIR / "out" / "monte_carlo"
SETTINGS_FILE_PRE = "OSLisfloodGloFASv5calibration_v1_SouthAmerica_ChilePreRunlong_term_run.xml"
SETTINGS_FILE = "OSLisfloodGloFASv5calibration_v1_SouthAmerica_ChileRunlong_term_run.xml"

# Create output directory
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Simulation period (for this example)
STEP_START = "02-01-1975 00:00"
STEP_END = "01/01/1979 00:00"

## Define Parameter Bounds

In [ ]:
# Parameter bounds for Monte Carlo sampling
# (name, min_value, max_value)
PARAM_BOUNDS = {
    "UpperZoneTimeConstant": (0.01, 40),
    "LowerZoneTimeConstant": (40, 1000),
    "GwPercValue": (0.01, 2),
    "LZThreshold": (0, 30),
    "b_Xinanjiang": (0.01, 5),
    "PowerPrefFlow": (0.5, 8),
    "SnowMeltCoef": (2.5, 6.5),
    "CalChanMan1": (0.5, 2),
    "CalChanMan2": (0.5, 5),
    "GwLoss": (0, 0.5),
    "TransSub": (0, 0.12)
}

print(f"Number of parameters: {len(PARAM_BOUNDS)}")
print(f"Monte Carlo samples: {N_SAMPLES}")
print(f"\nParameter bounds:")
for name, (min_val, max_val) in PARAM_BOUNDS.items():
    print(f"  {name}: [{min_val}, {max_val}]")

## Helper Functions

In [ ]:
def set_textvar(root, name, new_value, must_exist=True):
    """
    Modify the 'value' attribute of a <textvar> element in a LISFLOOD settings file.

    Parameters
    ----------
    root : xml.etree.ElementTree.Element
        Root element of the parsed XML tree (typically <lfsettings>).
    name : str
        Value of the 'name' attribute of the <textvar> to modify.
    new_value : str or float or int
        New value to assign to the 'value' attribute.
    must_exist : bool, optional
        If True (default), raise an error if the parameter is not found.
        If False, silently do nothing if the parameter does not exist.

    Returns
    -------
    bool
        True if the parameter was found and modified, False otherwise.
    """
    for tv in root.iter("textvar"):
        if tv.attrib.get("name") == name:
            tv.attrib["value"] = str(new_value)
            return True

    if must_exist:
        raise KeyError(f"textvar '{name}' not found in settings file")

    return False


def generate_random_params(param_bounds, n_samples, seed=None):
    """
    Generate random parameter sets within defined bounds.

    Parameters
    ----------
    param_bounds : dict
        Dictionary with parameter names as keys and (min, max) tuples as values.
    n_samples : int
        Number of parameter sets to generate.
    seed : int, optional
        Random seed for reproducibility.

    Returns
    -------
    list of dict
        List of parameter dictionaries.
    """
    if seed is not None:
        np.random.seed(seed)

    param_sets = []
    for i in range(n_samples):
        params = {}
        for name, (min_val, max_val) in param_bounds.items():
            params[name] = np.random.uniform(min_val, max_val)
        param_sets.append(params)

    return param_sets


def run_lisflood(settings_file, output_dir, step_start, step_end):
    """
    Run LISFLOOD model using command line.

    Parameters
    ----------
    settings_file : str
        Path to the settings XML file.
    output_dir : Path
        Directory for model output.
    step_start : str
        Start date/time.
    step_end : str
        End date/time.

    Returns
    -------
    int
        Return code (0 = success).
    """
    cmd = [
        "lisflood",
        settings_file,
        "--stepstart", step_start,
        "--stepend", step_end,
        "--outputdir", str(output_dir)
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.returncode

## Monte Carlo Calibration

In [ ]:
# Generate random parameter sets
np.random.seed(42)  # For reproducibility
param_sets = generate_random_params(PARAM_BOUNDS, N_SAMPLES, seed=42)

print(f"Generated {len(param_sets)} random parameter sets")
print("\nFirst 3 parameter sets:")
for i, params in enumerate(param_sets[:3]):
    print(f"\nSet {i+1}:")
    for name, value in params.items():
        print(f"  {name}: {value:.4f}")

In [ ]:
# Run Monte Carlo simulations
results = []

print("=" * 60)
print(f"Starting Monte Carlo Calibration ({N_SAMPLES} runs)")
print("=" * 60)

start_time = time.time()

for i, params in enumerate(param_sets):
    run_id = i + 1
    print(f"\n--- Run {run_id}/{N_SAMPLES} ---")
    
    # Create unique output directory for this run
    run_out_dir = OUT_DIR / f"run_{run_id:04d}"
    run_out_dir.mkdir(parents=True, exist_ok=True)
    
    # Copy and modify settings file
    settings_path = IN_DIR / SETTINGS_FILE
    settings_tree = ET.parse(settings_path)
    root = settings_tree.getroot()
    
    # Set simulation period
    set_textvar(root, "StepStart", STEP_START, must_exist=False)
    set_textvar(root, "StepEnd", STEP_END, must_exist=False)
    set_textvar(root, "PathOut", str(run_out_dir), must_exist=False)
    
    # Set calibration parameters
    for name, value in params.items():
        set_textvar(root, name, value, must_exist=False)
    
    # Write modified settings file
    run_settings_file = run_out_dir / "settings_mc.xml"
    settings_tree.write(run_settings_file)
    
    # Run LISFLOOD
    print(f"  Parameters: ", end="")
    print(", ".join([f"{k}={v:.2f}" for k, v in list(params.items())[:3]]) + "...")
    
    return_code = run_lisflood(str(run_settings_file), run_out_dir, STEP_START, STEP_END)
    
    if return_code == 0:
        print(f"  ✓ Run completed successfully")
        results.append({
            "run_id": run_id,
            "params": params,
            "return_code": return_code,
            "output_dir": run_out_dir
        })
    else:
        print(f"  ✗ Run failed with code {return_code}")
        results.append({
            "run_id": run_id,
            "params": params,
            "return_code": return_code,
            "output_dir": run_out_dir
        })

elapsed_time = time.time() - start_time
print("\n" + "=" * 60)
print(f"Monte Carlo Calibration Complete")
print(f"Total time: {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")
print(f"Successful runs: {sum(1 for r in results if r['return_code'] == 0)}/{N_SAMPLES}")
print("=" * 60)

## Results Summary

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame([
    {"run_id": r["run_id"], **r["params"], "return_code": r["return_code"]}
    for r in results
])

print("Results Summary:")
print(results_df.to_string(index=False))

# Save results to CSV
results_csv = OUT_DIR / "monte_carlo_results.csv"
results_df.to_csv(results_csv, index=False)
print(f"\nResults saved to: {results_csv}")

## Next Steps

This is a basic Monte Carlo framework. To make it a proper calibration:

1. **Load observed data** - Read the observed discharge time series
2. **Load simulated data** - Read the LISFLOOD output for each run
3. **Calculate objective function** - Compute NSE, KGE, or other metrics for each run
4. **Identify best parameters** - Find the parameter set with the highest objective function value
5. **Visualize results** - Plot parameter distributions, objective function values, etc.

For a more sophisticated approach, consider:
- Latin Hypercube Sampling for more efficient parameter space exploration
- Evolutionary algorithms (e.g., NSGA-II, CMA-ES) for optimization
- Sensitivity analysis to identify most influential parameters